
# 範例 : RNN Seq 2 Seq 加 attention NMT
***
- 本範例目標是藉由手刻一個 RNN Seq 2 Seq  並且實用在一個簡單的 中翻英的翻譯應用上 以增進學員對課程的了解
-- 實做 attention 在 RNN Seq 2 Seq 以更了解　課程內容
-- 應用 RNN Seq 2 Seq 加 attention 解決一個機器翻譯的問題

# [教學目標]
- 了解 最簡單的 attention 機制如何建立 
- 了解如何建立一個機器翻譯模型 之後 transformer 課程也會使用 

# [範例重點]

- 了解 最簡單的 attention 機制如何建立
- 了解如何建立一個機器翻譯模型 


In [1]:
!nvidia-smi

Thu Feb  4 07:23:22 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install torchtext==0.6.0

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.data import Field, BucketIterator, TabularDataset
from sklearn.model_selection import train_test_split
import csv

import numpy as np
import re
import random
import math
import time

import csv
import spacy


In [3]:
# Colab 進行matplotlib繪圖時顯示繁體中文
# 下載字體並命名taipei_sans_tc_beta.ttf，移至指定路徑
!wget -O taipei_sans_tc_beta.ttf https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_&export=download
!mv taipei_sans_tc_beta.ttf /usr/local/lib/python3.6/dist-packages/matplotlib//mpl-data/fonts/ttf

from matplotlib.font_manager import FontProperties
import matplotlib.pyplot as plt 
plt.style.use("seaborn-whitegrid")
import matplotlib.ticker as ticker
# 自定義字體變數
myfont = FontProperties(fname=r'/usr/local/lib/python3.6/dist-packages/matplotlib/mpl-data/fonts/ttf/taipei_sans_tc_beta.ttf')

--2021-02-04 07:23:45--  https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_
Resolving drive.google.com (drive.google.com)... 74.125.20.101, 74.125.20.139, 74.125.20.100, ...
Connecting to drive.google.com (drive.google.com)|74.125.20.101|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0k-9o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/h8b8r7miemhessan9abbvfsif9ks4ods/1612423425000/02847987870453524430/*/1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_ [following]
--2021-02-04 07:23:47--  https://doc-0k-9o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/h8b8r7miemhessan9abbvfsif9ks4ods/1612423425000/02847987870453524430/*/1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_
Resolving doc-0k-9o-docs.googleusercontent.com (doc-0k-9o-docs.googleusercontent.com)... 74.125.28.132, 2607:f8b0:400e:c04::84
Connecting to doc-0k-9o-docs.googleusercontent.com (doc-0k-9o-docs.googleusercontent.com)|74.125.28.1

先連接自己的GOOGLE DRIVE 為了要儲存資料和訓練模型

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


機器翻譯的訓練資料來源是從 https://www.manythings.org/anki/ 下載下來的中英翻譯文本

Chinese (Mandarin) - English --> cmn-eng.zip (23610)

麻煩大家從網路上下載檔案 解壓後檔案名稱叫做 cmn.txt 請放在一個叫做 /data/的子目錄下

我們先把他從檔案讀出來

In [8]:
!wget https://www.manythings.org/anki/cmn-eng.zip

--2021-02-04 07:27:18--  https://www.manythings.org/anki/cmn-eng.zip
Resolving www.manythings.org (www.manythings.org)... 172.67.173.198, 104.21.55.222, 2606:4700:3031::6815:37de, ...
Connecting to www.manythings.org (www.manythings.org)|172.67.173.198|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1062383 (1.0M) [application/zip]
Saving to: ‘cmn-eng.zip’

cmn-eng.zip         100%[===================>]   1.01M  2.05MB/s    in 0.5s    

2021-02-04 07:27:19 (2.05 MB/s) - ‘cmn-eng.zip’ saved [1062383/1062383]



In [9]:
!unzip cmn-eng.zip

Archive:  cmn-eng.zip
  inflating: cmn.txt                 
  inflating: _about.txt              


In [11]:
!mkdir data
!mv ./zip/cmn.txt ./data

In [13]:
data_dir = './data/'
lines = open(data_dir + 'cmn.txt' , encoding='utf-8').read().strip().split('\n')
trnslt_pairs = [[s for s in l.split('\t')] for l in lines ]
print ("Sample: " , trnslt_pairs[1000][0:2] )
print ("Total records:" , len(trnslt_pairs))

Sample:  ['He was drowned.', '他被淹死了。']
Total records: 24360


# 在這邊開始準備資料

## 為了簡化我們的問題在這邊我們特別刪除了不是以"你我他她"這幾個中文字開頭的翻譯句

## 在剩下的 13039 筆資料中 我們保留 大約各一千筆給 dev 資料集和 testing 資料集

## 之後我們把三個資料集以 csv的格式 儲存在 /data/ 目錄下

In [14]:

# create train and validation set
# remove sentence not with ['我','你','他','她']

trnslt_pairs = [pair for pair in trnslt_pairs if pair[1][0] in ['我','你','他','她']]
print ("Total records after filtering :" , len(trnslt_pairs))
train, test = train_test_split(trnslt_pairs, test_size=0.08)
train, val = train_test_split(train, test_size=0.09)
print ("training data:{} , develop data: {} , testing data: {}".format(len(train),len(val),len(test)))
    
def write_csv(trn_data, file_path ):
    with open(file_path ,'w', newline='', encoding='utf-8') as fout:
        writer = csv.writer (fout)
        for itm in trn_data: 
            writer.writerow ([itm[0],itm[1]])
            
file_path = data_dir + 'train.csv'
write_csv(train, file_path )

file_path = data_dir + 'val.csv'
write_csv(val, file_path )
    
file_path = data_dir + 'test.csv'
write_csv(test, file_path )

Total records after filtering : 13420
training data:11234 , develop data: 1112 , testing data: 1074


# 使用我們之前介紹過的的 torchtext 來準備訓練資料

## 如果對torchtext 不太熟習的人可以回去複習一下

## 這邊有用到一個有名的 NLP套件 spacy 幫我們做 英文tokenize

In [15]:
# 下載 spacy 的英文模型 幫我們做tokenize
spacy_eng = spacy.load('en_core_web_sm')
def tokenize_eng(text):
  #清除不需要的字符
  text = re.sub(r"([.!?])", r" \1", text)
  return [tok.text for tok in spacy_eng.tokenizer(text)]

TRG = Field(tokenize = tokenize_eng, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)



def tokenize_cmn(text):
  #去掉非中文字元
  regex = re.compile(r'[^\u4e00-\u9fa5A-Za-z0-9]')
  text = regex.sub(' ', text)

  return [word for word in text if word.strip()]
    

SRC = Field(tokenize = tokenize_cmn, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True, 
            include_lengths = True)


# 使用torchtext TabularDataset 讀資料

## 並且分別建立 src and target 的 字元表

In [16]:
train_dataset, dev_dataset, test_dataset = TabularDataset.splits(
    path = data_dir , format = 'csv', skip_header = True,
    train='train.csv', validation='val.csv', test='test.csv',
    fields=[
        ('trg', TRG),
        ('src', SRC)
    ]
)
SRC.build_vocab(train_dataset, min_freq = 1)
TRG.build_vocab(train_dataset, min_freq = 1)

print ("中文語料的字元表長度: " , len(SRC.vocab) , ", 英文的字元表長度: " ,len(TRG.vocab))

中文語料的字元表長度:  2683 , 英文的字元表長度:  4082


# 建立 train valiate test 資料 iterator 

In [17]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_dataset, dev_dataset, test_dataset), 
     batch_size = BATCH_SIZE,
     sort_within_batch = True,
     sort_key = lambda x : len(x.src),
     device = device)

# 這邊是我們的重點 attention 層

## 這邊實作的點積 attention

In [18]:
class Attention(nn.Module):
  def __init__(self, enc_hid_dim, dec_hid_dim):
    super().__init__()

  def forward(self, hidden, encoder_outputs, mask):
    # hidden bz , dec_hid_dim
    # encoder_outputs src len, bz , enc_hid_dim x 2
    # mask bz , src len
    
    batch_size = encoder_outputs.shape[1]
    src_len = encoder_outputs.shape[0]

    hidden = hidden.unsqueeze(1) 
    # hidden unsqueeze bz , 1 , dec_hid_dim

    attention = torch.matmul( hidden , encoder_outputs.permute(1, 2, 0)   )
    # attention bz, 1 , src len
    
    attention = attention.squeeze(1)
    # squeeze bz , src len

    attention = attention.masked_fill(mask == 0, -1e10)

    return F.softmax(attention, dim = 1)

# 這是一個簡單的 以GRU 為底層 bi-directional encoder 

## 有關RNN部分的概念請參照前章

## 我們這邊不多做說明

In [19]:
class RNNEncoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        # 雙向 ＧＲＵ encoder 
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, src_len):
        
        #src shape [src len, batch size]
        #src_len shape [batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded shape [src len, batch size, emb dim]
                
        # 使用pack_padded_sequence 來壓縮序列        
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, src_len)
                
        packed_outputs, hidden = self.rnn(packed_embedded)

        # 使用 pad_packed_sequence 用來展開序列成原本形狀的      
        outputs, _ = nn.utils.rnn.pad_packed_sequence(packed_outputs) 
            
            
        #outputs shape [src len, batch size, hid dim * num directions]
        #hidden shape [n layers * num directions, batch size, hid dim]
        
        #hidden 堆疊 [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs 是最後一層 
        
        #hidden [-2, :, : ] 是最後一層 forwards RNN 
        #hidden [-1, :, : ] 是最後一層 backwards RNN
        
        # hidden 是最後再過一層 dense layer
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        
        #outputs shape [src len, batch size, enc hid dim * 2]
        #hidden shape [batch size, dec hid dim]
        
        return outputs, hidden

# 這是一個以GRU 為底層的 RNN decoder.

## 請注意最後一個參數 attention

## 這是我們的重點 attention, 讓我們在這邊好好的觀察 attention 怎樣接受輸入

## 並且怎樣被使用

In [20]:
class RNNDecoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        # 單向 ＧＲＵ decoder 
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs, mask):
             
        #input shape [batch size]
        #hidden shape [batch size, dec hid dim]
        #encoder_outputs shape [src len, batch size, enc hid dim * 2]
        #mask shape [batch size, src len]
        
        input = input.unsqueeze(0)
        
        #input shape [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded shape [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs, mask)
                
        #a shape [batch size, src len]
        
        a = a.unsqueeze(1)
        
        #a shape [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs shape [batch size, src len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)
        
        #weighted shape [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted shape [1, batch size, enc hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        
        #rnn_input shape [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        #output shape [seq len, batch size, dec hid dim * n directions]
        #hidden shape [n layers * n directions, batch size, dec hid dim]
        
        #seq len, n layers and n directions will always be 1 in this decoder, therefore:
        #output shape [1, batch size, dec hid dim]
        #hidden shape [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        
        #prediction shape [batch size, output dim]
        
        return prediction, hidden.squeeze(0), a.squeeze(1)

# 這邊是我們的 seq 2 seq 模組

## 主要的概念之前的章節有介紹

## 這邊只有簡單的說明

In [21]:
class Seq2SeqATTN(nn.Module):
    def __init__(self, encoder, decoder, src_pad_idx, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.device = device
        
    def create_mask(self, src):
        mask = (src != self.src_pad_idx).permute(1, 0)
        return mask
        
    def forward(self, src, src_len, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #src_len = [batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
                    
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src, src_len)
                
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        mask = self.create_mask(src)

        #mask = [batch size, src len]
                
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state, all encoder hidden states 
            #  and mask
            #receive output tensor (predictions) and new hidden state
            output, hidden, _ = self.decoder(input, hidden, encoder_outputs, mask)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
            
        return outputs

# 在這邊先設定關於我們模型的重要參數

## 然後將模型建立起來並且初始化權重

## 可以觀察一下我們模型結構

In [22]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 256 # 注意 encoder hidden layer 設定 必須為 dec 的一半 
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

LEARNING_RATE = 0.002

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = RNNEncoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = RNNDecoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)
model = Seq2SeqATTN(enc, dec, SRC_PAD_IDX, device).to(device)

optimizer = optim.Adam(model.parameters(),lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)


def initial_mdl_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(initial_mdl_weights)
print ("模型全部參數量: {:10,d} ".format(sum(p.numel() for p in model.parameters())))
model

模型全部參數量:  9,982,194 


Seq2SeqATTN(
  (encoder): RNNEncoder(
    (embedding): Embedding(2683, 256)
    (rnn): GRU(256, 256, bidirectional=True)
    (fc): Linear(in_features=512, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): RNNDecoder(
    (attention): Attention()
    (embedding): Embedding(4082, 256)
    (rnn): GRU(768, 512)
    (fc_out): Linear(in_features=1280, out_features=4082, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

# 這邊是訓練和evalutate 

In [23]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src, src_len = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, src_len.cpu() , trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src, src_len = batch.src
            trg = batch.trg

            output = model(src, src_len.cpu(), trg, 0) #turn off teacher forcing
            
            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

# 訓練設定


In [24]:
MAX_EPOCHS = 20
CLIP = 1
model_dir =  './data/'
best_valid_loss = 9999999

for epoch in range(MAX_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    torch.save(model.state_dict(), model_dir + 'model-{}.pt'.format(epoch))
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), model_dir + 'best-model.pt')
   
    print ("Epoch {} training time: {:.2f} sec Training Loss: {:.3f} , Valiation Loss: {:.3f}".format( epoch , end_time - start_time , train_loss , valid_loss))

Epoch 0 training time: 5.99 sec Training Loss: 5.327 , Valiation Loss: 5.079
Epoch 1 training time: 5.68 sec Training Loss: 4.432 , Valiation Loss: 4.647
Epoch 2 training time: 5.70 sec Training Loss: 4.019 , Valiation Loss: 4.474
Epoch 3 training time: 5.71 sec Training Loss: 3.547 , Valiation Loss: 4.038
Epoch 4 training time: 5.69 sec Training Loss: 3.163 , Valiation Loss: 3.815
Epoch 5 training time: 5.64 sec Training Loss: 2.733 , Valiation Loss: 3.661
Epoch 6 training time: 5.66 sec Training Loss: 2.364 , Valiation Loss: 3.582
Epoch 7 training time: 5.65 sec Training Loss: 2.060 , Valiation Loss: 3.614
Epoch 8 training time: 5.65 sec Training Loss: 1.787 , Valiation Loss: 3.525
Epoch 9 training time: 5.69 sec Training Loss: 1.586 , Valiation Loss: 3.567
Epoch 10 training time: 5.83 sec Training Loss: 1.412 , Valiation Loss: 3.569
Epoch 11 training time: 5.72 sec Training Loss: 1.224 , Valiation Loss: 3.642
Epoch 12 training time: 5.65 sec Training Loss: 1.058 , Valiation Loss: 3.

# 為了作業我們要先將模型儲存
## 將 vocabulaty 也一併存下


In [25]:
# Save SRC and TRG vocab
torch.save(SRC.vocab, model_dir + 'SRC_vocab.pt')
torch.save(TRG.vocab, model_dir + 'TRG_vocab.pt')

# Evaluate Loss of testing

In [26]:
model.load_state_dict(torch.load(model_dir + 'best-model.pt'))
#model.load_state_dict(torch.load(model_dir + 'model-7.pt'))
test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.768 | Test PPL:  43.310 |


# 這邊是執行真正的翻譯工作的函式 

## 為了讓我們之後可以展現結果和計算BLEU值

In [27]:
def translate_sentence(sentence, src_field, trg_field, model, device, max_len = 50):

    model.eval()
        
    #if isinstance(sentence, str):
    #    nlp = spacy_en = spacy.load('en_core_web_sm')
    #    tokens = [token.text.lower() for token in spacy_en(sentence)]
    #else:
    #    tokens = [token.lower() for token in sentence]

    tokens = [token.lower() for token in sentence]
        
    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
        
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    src_len = torch.LongTensor([len(src_indexes)]).to(device)
    
    with torch.no_grad():
        encoder_outputs, hidden = model.encoder(src_tensor, src_len.cpu())

    mask = model.create_mask(src_tensor)
        
    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    attentions = torch.zeros(max_len, 1, len(src_indexes)).to(device)
    
    for i in range(max_len):

        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)
                
        with torch.no_grad():
            output, hidden, attention = model.decoder(trg_tensor, hidden, encoder_outputs, mask)

        attentions[i] = attention
            
        pred_token = output.argmax(1).item()
        
        trg_indexes.append(pred_token)

        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break
    
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]
    
    return trg_tokens[1:], attentions[:len(trg_tokens)-1]

# 選擇一句來看翻譯結果

In [28]:
example_idx =520

src = vars(train_dataset.examples[example_idx])['src']
trg = vars(train_dataset.examples[example_idx])['trg']

print(f'src = {src}')
print(f'trg = {trg}')

translation, attention = translate_sentence(src, SRC, TRG, model, device)

print(f'predicted trg = {translation}')

src = ['我', '受', '夠', '了', '在', '餐', '館', '吃', '飯']
trg = ['i', "'m", 'fed', 'up', 'with', 'eating', 'in', 'restaurants', '.']
predicted trg = ['i', 'eat', 'tea', 'for', 'eating', 'in', 'eating', '.', '<eos>']


# BLEU score implementation
## BLEU是雙語評估 (Bilingual Evaluation Understudy) 的縮寫。 它測量翻譯質量的方法是與一組選定的人工翻譯結果進行比較，並測量命中該組人工翻譯結果的機械翻譯結果的準確性。 根據用戶的需要和對文本的依賴性，可以用比單個詞彙表（1-gram 或 uni-gram）更長的長度來測量此準確性，例如選擇2-4個字長（2-4 gram）而不會丟失 單詞之間的順序，以衡量機械翻譯對人類翻譯的準確性。

In [29]:
from torchtext.data.metrics import bleu_score

def calculate_bleu(data, src_field, trg_field, model, device, max_len = 50):
    
    trgs = []
    pred_trgs = []
    
    for datum in data:
        
        src = vars(datum)['src']
        trg = vars(datum)['trg']
        
        pred_trg, _ = translate_sentence(src, src_field, trg_field, model, device, max_len)
        
        #cut off <eos> token
        pred_trg = pred_trg[:-1]
        
        pred_trgs.append(pred_trg)
        trgs.append([trg])
        
    return bleu_score(pred_trgs, trgs)

ModuleNotFoundError: ignored

In [ ]:
bleu_score = calculate_bleu(test_dataset, SRC, TRG, model, device)

print(f'BLEU score = {bleu_score*100:.2f}')

In [30]:
!mkdir ./drive/MyDrive/0204_model
! mv ./data/* ./drive/MyDrive/0204_model/